## datetime 제외 모든 테이블 가져오기

In [1]:
import pymysql
import json
import os
from datetime import datetime

def get_connection():
    host = input("DB 호스트: ")
    user = input("DB 사용자명: ")
    password = input("DB 비밀번호: ")
    db = input("DB 이름: ")
    port = int(input("포트 (기본 3306): ") or "3306")

    conn = pymysql.connect(
        host=host,
        user=user,
        password=password,
        db=db,
        port=port,
        charset='utf8'
    )
    print(f"\n>>> DB 연결 성공: {db}\n")
    return conn

def get_table_list(cursor, db_name):
    cursor.execute(f"SHOW TABLES FROM `{db_name}`")
    return [row[0] for row in cursor.fetchall()]

def get_datetime_column(cursor, db_name, table):
    cursor.execute("""
        SELECT COLUMN_NAME, DATA_TYPE
        FROM information_schema.columns
        WHERE table_schema = %s AND table_name = %s
    """, (db_name, table))

    datetime_columns = [row[0] for row in cursor.fetchall()
                        if row[1].lower() in ("datetime", "timestamp")]

    if not datetime_columns:
        return None

    return datetime_columns[0]

def main():
    conn = get_connection()
    cursor = conn.cursor()
    db_name = conn.db.decode() if isinstance(conn.db, bytes) else conn.db

    tables = get_table_list(cursor, db_name)
    no_datetime_tables = {}

    for table in tables:
        datetime_col = get_datetime_column(cursor, db_name, table)
        if not datetime_col:
            print(f"추출 대상: {table} (datetime 또는 timestamp 컬럼 없음)")
            no_datetime_tables[table] = "datetime/timestamp 컬럼 없음"

    cursor.close()
    conn.close()

    output = json.dumps(no_datetime_tables, indent=2, ensure_ascii=False)

    print("\n결과:")
    print(output)

    filename = f"no_datetime_tables_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(output)

    print(f"\n✔ 결과 파일 저장 완료: {filename}")
    print(f"파일 경로: {os.path.abspath(filename)}")

if __name__ == "__main__":
    main()



>>> DB 연결 성공: sc301web

추출 대상: MDCL_RSV (datetime 또는 timestamp 컬럼 없음)
추출 대상: Surveys (datetime 또는 timestamp 컬럼 없음)
추출 대상: _tiaraEvent (datetime 또는 timestamp 컬럼 없음)
추출 대상: contents (datetime 또는 timestamp 컬럼 없음)
추출 대상: crontab (datetime 또는 timestamp 컬럼 없음)
추출 대상: customer (datetime 또는 timestamp 컬럼 없음)
추출 대상: insr_info (datetime 또는 timestamp 컬럼 없음)
추출 대상: md_crawling (datetime 또는 timestamp 컬럼 없음)
추출 대상: md_crm_code (datetime 또는 timestamp 컬럼 없음)
추출 대상: md_db_trans (datetime 또는 timestamp 컬럼 없음)
추출 대상: md_db_trans_tm (datetime 또는 timestamp 컬럼 없음)
추출 대상: md_ld_consult_memo (datetime 또는 timestamp 컬럼 없음)
추출 대상: md_ld_food_log (datetime 또는 timestamp 컬럼 없음)
추출 대상: md_ld_log (datetime 또는 timestamp 컬럼 없음)
추출 대상: md_menu_log (datetime 또는 timestamp 컬럼 없음)
추출 대상: md_modus_deli_cost (datetime 또는 timestamp 컬럼 없음)
추출 대상: md_modus_ex_rate (datetime 또는 timestamp 컬럼 없음)
추출 대상: mdcl_rsv (datetime 또는 timestamp 컬럼 없음)
추출 대상: meal_evaluation_details (datetime 또는 timestamp 컬럼 없음)
추출 대상: meal_evaluation_food_gro

## datetime 제외 테이블 값 가져오기

In [3]:
import pymysql
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import json
import os

START_DATE = datetime(2022, 1, 1)
END_DATE = datetime(2025, 6, 1)

def get_connection():
    host = input("DB 호스트: ")
    user = input("DB 사용자명: ")
    password = input("DB 비밀번호: ")
    db = input("DB 이름: ")
    port = int(input("포트 (기본 3306): ") or "3306")

    conn = pymysql.connect(
        host=host,
        user=user,
        password=password,
        db=db,
        port=port,
        charset='utf8',
        connect_timeout=60,
        read_timeout=300,
        write_timeout=300
    )
    print(f"\n>>> DB 연결 성공: {db}\n")
    return conn

def get_table_list(cursor, db_name):
    cursor.execute(f"SHOW TABLES FROM `{db_name}`")
    return [row[0] for row in cursor.fetchall()]

def has_datetime_column(cursor, db_name, table):
    cursor.execute("""
        SELECT DATA_TYPE
        FROM information_schema.columns
        WHERE table_schema = %s AND table_name = %s
    """, (db_name, table))
    return any(row[0].lower() in ("datetime", "timestamp") for row in cursor.fetchall())

def get_avg_row_length(cursor, db_name, table):
    cursor.execute("""
        SELECT AVG_ROW_LENGTH
        FROM information_schema.tables
        WHERE table_schema = %s AND table_name = %s
    """, (db_name, table))
    result = cursor.fetchone()
    return result[0] if result and result[0] else 0

def get_row_count(cursor, db_name, table):
    cursor.execute(f"SELECT COUNT(*) FROM `{db_name}`.`{table}`")
    return cursor.fetchone()[0]

def date_range_by_unit(start, end, unit):
    ranges = []
    current = start
    while current < end:
        if unit == 'week':
            next_point = current + timedelta(days=7)
            label = f"{current:%Y-W%U}"
        elif unit == 'month':
            next_point = current + relativedelta(months=1)
            label = f"{current:%Y-%m}"
        elif unit == 'year':
            next_point = current + relativedelta(years=1)
            label = f"{current:%Y}"
        else:
            raise ValueError("단위 오류")
        ranges.append((label, current, next_point))
        current = next_point
    return ranges

def estimate_evenly_distributed_usage(row_count, avg_len, ranges):
    total_mb = row_count * avg_len / 1024 / 1024
    per_range_mb = total_mb / len(ranges) if ranges else 0
    return {label: f"{per_range_mb:,.3f}" for label, _, _ in ranges}

def safe_date_format(date_obj):
    return date_obj.strftime('%Y-%m-%d') if hasattr(date_obj, 'strftime') else str(date_obj)

def main():
    conn = get_connection()
    cursor = conn.cursor()
    db_name = conn.db.decode() if isinstance(conn.db, bytes) else conn.db

    tables = get_table_list(cursor, db_name)
    final_result = {}

    for table in tables:
        if has_datetime_column(cursor, db_name, table):
            continue  # datetime 컬럼이 있으면 스킵

        print(f"처리 중 (datetime 없음): {table}")

        avg_len = get_avg_row_length(cursor, db_name, table)
        if avg_len == 0:
            print(f"건너뜀: {table} (AVG_ROW_LENGTH = 0)")
            continue

        row_count = get_row_count(cursor, db_name, table)
        if row_count == 0:
            print(f"건너뜀: {table} (row 없음)")
            continue

        table_result = {
            "startDate": safe_date_format(START_DATE),
            "endDate": safe_date_format(END_DATE),
            "week": {},
            "month": {},
            "year": {}
        }

        for unit in ['week', 'month', 'year']:
            ranges = date_range_by_unit(START_DATE, END_DATE, unit)
            usage = estimate_evenly_distributed_usage(row_count, avg_len, ranges)
            table_result[unit] = usage

        final_result[table] = table_result

    output = json.dumps(final_result, indent=2, ensure_ascii=False)
    print("\n결과값:")
    print(output)

    filename = f"db_no_datetime_estimate_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(output)

    print(f"\n완료! 결과 파일: {filename}")
    print(f"파일 경로: {os.path.abspath(filename)}")

    cursor.close()
    conn.close()

if __name__ == "__main__":
    main()



>>> DB 연결 성공: sc301web

처리 중 (datetime 없음): MDCL_RSV
건너뜀: MDCL_RSV (AVG_ROW_LENGTH = 0)
처리 중 (datetime 없음): Surveys
처리 중 (datetime 없음): _tiaraEvent
처리 중 (datetime 없음): contents
처리 중 (datetime 없음): crontab
처리 중 (datetime 없음): customer
처리 중 (datetime 없음): insr_info
건너뜀: insr_info (AVG_ROW_LENGTH = 0)
처리 중 (datetime 없음): md_crawling
처리 중 (datetime 없음): md_crm_code
처리 중 (datetime 없음): md_db_trans
처리 중 (datetime 없음): md_db_trans_tm
처리 중 (datetime 없음): md_ld_consult_memo
처리 중 (datetime 없음): md_ld_food_log
처리 중 (datetime 없음): md_ld_log
처리 중 (datetime 없음): md_menu_log
처리 중 (datetime 없음): md_modus_deli_cost
처리 중 (datetime 없음): md_modus_ex_rate
처리 중 (datetime 없음): mdcl_rsv
건너뜀: mdcl_rsv (AVG_ROW_LENGTH = 0)
처리 중 (datetime 없음): meal_evaluation_details
처리 중 (datetime 없음): meal_evaluation_food_groups
처리 중 (datetime 없음): post_test
처리 중 (datetime 없음): product
처리 중 (datetime 없음): rcpt_dtal_cntn
건너뜀: rcpt_dtal_cntn (AVG_ROW_LENGTH = 0)
처리 중 (datetime 없음): rcpt_info
건너뜀: rcpt_info (AVG_ROW_LENGTH = 0)


## datetime 없는 테이블 엑셀 내보내기

In [1]:
import json
import pandas as pd
import os

def flatten_json_to_dataframe(json_data):
    all_months = set()
    all_years = set()
    table_rows = []

    for table_name, table_data in json_data.items():
        if table_name == "skipped":
            continue

        months = table_data.get("month", {})
        years = table_data.get("year", {})

        all_months.update(months.keys())
        all_years.update(years.keys())

    all_months = sorted(all_months)
    all_years = sorted(all_years)
    columns = ["table"] + all_months + all_years + ["총합 (MB)"]

    for table_name, table_data in json_data.items():
        if table_name == "skipped":
            continue

        row = {"table": table_name}
        total = 0.0

        # 월 단위
        for m in all_months:
            val = table_data.get("month", {}).get(m, "")
            row[m] = val or ""

        # 연 단위 + 총합 계산
        for y in all_years:
            val = table_data.get("year", {}).get(y, "")
            row[y] = val or ""
            if val:
                try:
                    total += float(val.replace(",", ""))
                except ValueError:
                    pass

        row["총합 (MB)"] = f"{total:,.3f}" if total else ""
        table_rows.append(row)

    return pd.DataFrame(table_rows, columns=columns)

def main():
    # filename = "db_size_estimate_20250619.json"  # 파일명
    filename = "db_no_datetime_estimate_20250620_161744.json"
    if not os.path.exists(filename):
        print(f"파일이 존재하지 않습니다: {filename}")
        return

    with open(filename, "r", encoding="utf-8") as f:
        json_data = json.load(f)

    df = flatten_json_to_dataframe(json_data)

    output_filename = "db_not_size_summary.xlsx"
    df.to_excel(output_filename, index=False)

    print(f"\n✔ 엑셀 파일 생성 완료: {output_filename}")
    print(f"경로: {os.path.abspath(output_filename)}")

if __name__ == "__main__":
    main()



✔ 엑셀 파일 생성 완료: db_not_size_summary.xlsx
경로: /Users/sc301/Desktop/yhy/project/lagacy_db/db_not_size_summary.xlsx


## datetime 없는 테이블 line chart

In [ ]:
import json
import pandas as pd
from datetime import datetime
import plotly.express as px
import os

def load_json(filename):
    with open(filename, "r", encoding="utf-8") as f:
        return json.load(f)

def preprocess_data(data):
    records = []

    for table_name, table_data in data.items():
        if table_name == "skipped":
            continue

        year_data = table_data.get("year", {})
        for year, size in year_data.items():
            try:
                size_mb = float(size.replace(",", ""))
                records.append({
                    "테이블": table_name,
                    "연도": int(year),
                    "용량(MB)": size_mb
                })
            except ValueError:
                continue

    df = pd.DataFrame(records)
    df = df.sort_values(by=["테이블", "연도"])

    # 🔧 연도를 문자열로 변환하여 x축을 카테고리로 인식하게 함
    df["연도"] = df["연도"].astype(str)

    return df

def save_to_excel(df, output_path):
    pivot_df = df.pivot(index="연도", columns="테이블", values="용량(MB)")
    pivot_df = pivot_df.fillna("")
    pivot_df.to_excel(output_path, index=True)
    print(f"엑셀 파일 저장 완료: {os.path.abspath(output_path)}")

def save_plot(df, output_path):
    fig = px.line(
        df,
        x="연도",
        y="용량(MB)",
        color="테이블",
        markers=True,
        title="테이블별 연도별 용량 추이 (MB)"
    )
    fig.write_html(output_path)
    print(f"차트 HTML 파일 저장 완료: {os.path.abspath(output_path)}")

def main():
    input_json = "db_no_datetime_estimate_20250620_161744.json"  # JSON 파일명

    if not os.path.exists(input_json):
        print(f"파일이 존재하지 않습니다: {input_json}")
        return

    data = load_json(input_json)
    df = preprocess_data(data)

    if df.empty:
        print("처리할 연도 데이터가 없습니다.")
        return

    excel_output = "db_not_yearly_size_summary.xlsx"
    chart_output = "db_not_yearly_size_chart.html"

    save_to_excel(df, excel_output)
    save_plot(df, chart_output)

if __name__ == "__main__":
    main()


엑셀 파일 저장 완료: /Users/sc301/Desktop/yhy/project/lagacy_db/db_not_yearly_size_summary.xlsx
차트 HTML 파일 저장 완료: /Users/sc301/Desktop/yhy/project/lagacy_db/db_not_yearly_size_chart.html
